In [15]:
import numpy as np

In [16]:
class stock():
    def __init__(self, s0, r, sigma, T, n, model = 'gbm'):
        self.s0 = s0
        self.r = r
        self.T = T
        self.n = n
        self.dt = T/n
        self.model = model
        self.sigma = sigma

    def vol(self, sigma):
        if self.model == 'gbm':
            return np.array([sigma] * self.n)
        elif self.model == 'heston':
            # Use the Heston volatility path
            vol_path = self.vol(self.sigma)
            innovations = np.random.normal(0, 1, self.n)
            stock_prices = np.zeros(self.n)
            stock_prices[0] = self.s0

            for i in range(1, self.n):
                stock_prices[i] = stock_prices[i-1] * np.exp(
                    (self.r - 0.5 * vol_path[i]**2) * self.dt + vol_path[i] * np.sqrt(self.dt) * innovations[i]
                )
            return stock_prices # Implement Heston model volatility here


    def heston_model_sim(S0, v0, rho, kappa, theta, sigma,T, N, M):

        """
        Inputs:
         - S0, v0: initial parameters for asset and variance
         - rho   : correlation between asset returns and variance
         - kappa : rate of mean reversion in variance process
         - theta : long-term mean of variance process
         - sigma : vol of vol / volatility of variance process
         - T     : time of simulation
         - N     : number of time steps
         - M     : number of scenarios / simulations

        Outputs:
        - asset prices over time (numpy array)
        - variance over time (numpy array)
        """
        # initialise other parameters
        dt = T/N
        mu = np.array([0,0])
        cov = np.array([[1,rho],
                        [rho,1]])

        # arrays for storing prices and variances
        S = np.full(shape=(N+1,M), fill_value=S0)
        v = np.full(shape=(N+1,M), fill_value=v0)

        # sampling correlated brownian motions under risk-neutral measure
        Z = np.random.multivariate_normal(mu, cov, (N,M))

        for i in range(1,N+1):
            S[i] = S[i-1] * np.exp( (r - 0.5*v[i-1])*dt + np.sqrt(v[i-1] * dt) * Z[i-1,:,0] )
            v[i] = np.maximum(v[i-1] + kappa*(theta-v[i-1])*dt + sigma*np.sqrt(v[i-1]*dt)*Z[i-1,:,1],0)

        return S, v

    def simulate(self):
        innovations = np.random.normal(0, 1, self.n)
        stock_prices = np.zeros(self.n)
        stock_prices[0] = self.s0

        for i in range(1, self.n):
            stock_prices[i] = stock_prices[i-1] * np.exp((self.r - 0.5 * self.sigma**2) * self.dt + self.sigma * np.sqrt(self.dt) * innovations[i])
        return stock_prices

    def option_price(self, K):
        stock_prices = self.simulate()
        payoff = np.maximum(stock_prices[-1] - K, 0)
        return np.exp(-self.r * self.T) * np.mean(payoff)


In [17]:
class simulation():

    import numpy as np
    import matplotlib.pyplot as plt

    # Parameters for simulation
    T_steps = 50         # Number of time steps (T)
    K_paths = 1000       # Number of Monte Carlo paths (K)
    T_total = 1.0        # Total time horizon (years)
    dt = T_total / T_steps
    S0 = 100             # Initial stock price
    r = 0.05             # Risk-free rate
    sigma = 0.2          # Volatility
    strike = 100         # Strike price (Z in pseudocode)
    lambda_param = 0.5   # λ parameter

    def simulate_stock_prices(S0, r, sigma, T_steps, K_paths, dt):
      """
      Simulate stock prices using a geometric Brownian motion.
      Returns an array S of shape (T_steps+1, K_paths).
      """
      S = np.zeros((T_steps + 1, K_paths))
      S[0] = S0
      for t in range(1, T_steps + 1):
          z = np.random.standard_normal(K_paths)
          S[t] = S[t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
      return S

    # Simulate the stock paths
    S = simulate_stock_prices(S0, r, sigma, T_steps, K_paths, dt)

    # %% [code]
    # Compute the state variable X.
    # For example, one may take X as the log of S (common in QLBS literature).
    X = np.log(S)

    # %% [code]
    # Define N basis functions; here we use a simple polynomial basis (constant, linear, quadratic)
    def basis_function_1(x):
        return np.ones_like(x)

    def basis_function_2(x):
        return x

    def basis_function_3(x):
        return x**2

    basis_functions = [basis_function_1, basis_function_2, basis_function_3]
    N_basis = len(basis_functions)

    # Create the feature matrix phi with dimensions (T_steps+1, K_paths, N_basis)
    phi = np.zeros((T_steps + 1, K_paths, N_basis))
    for t in range(T_steps + 1):
        for n, func in enumerate(basis_functions):
            phi[t, :, n] = func(X[t])

    # %% [code]
    # Initialize arrays for the variables computed in the backward recursion
    # a_star, Pi, R_star, and Q_star each have shape (T_steps+1, K_paths)
    a_star = np.zeros((T_steps + 1, K_paths))
    Pi = np.zeros((T_steps + 1, K_paths))
    R_star = np.zeros((T_steps + 1, K_paths))
    Q_star = np.zeros((T_steps + 1, K_paths))

    # Terminal conditions at t = T_steps
    # Compute the option payoff: max(strike - S_T, 0)
    Pi[T_steps] = np.maximum(strike - S[T_steps], 0)
    # Center the terminal portfolio (subtracting the mean)
    Pi_hat_T = Pi[T_steps] - np.mean(Pi[T_steps])
    # Set terminal action to zero
    a_star[T_steps] = 0
    # Terminal risk measure (here using the variance of the payoff; note that var is a scalar)
    R_star[T_steps] = -lambda_param * np.var(Pi[T_steps])
    # Terminal Q-value (again, note that the λ·Var term is constant across paths)
    Q_star[T_steps] = -Pi[T_steps] - lambda_param * np.var(Pi[T_steps])

    # %% [code]
    # Backward recursion from t = T_steps-1 to t = 0
    for t in range(T_steps - 1, -1, -1):
        # === Compute a_star[t] as in (44) ===
        # Placeholder: in a full implementation, you would estimate a regression of the continuation value
        # on the features phi[t]. Here we set it to zero.
        a_star[t] = 0  # Replace with actual computation using phi[t]

        # === Compute Pi[t] as in (29) ===
        # Placeholder: here we mimic the terminal payoff but an actual update rule may be more complex.
        Pi[t] = np.maximum(strike - S[t], 0)  # Replace with your QLBS update rule

        # === Compute R_star[t] as in (41) ===
        # Placeholder: you might compute a risk measure update here.
        R_star[t] = R_star[t + 1]  # Replace with actual computation

        # === Compute Q_star[t] as in (45) ===
        # Placeholder: combine the immediate cost and risk measure.
        Q_star[t] = -Pi[t] - lambda_param * np.var(Pi[t])  # Replace with the proper formula

    # %% [code]
    # Calculate the QLBS option price at t = 0:
    # QLBS_price = - (1/K_paths) * sum_{k=1}^{K_paths} Q_star[0, k]
    QLBS_price = -np.mean(Q_star[0])
    print("QLBS Option Price at t=0:", QLBS_price)


QLBS Option Price at t=0: -0.0


In [18]:
import numpy as np

class stock():
    def __init__(self, s0, r, sigma, T, n, model='gbm'):
        self.s0 = s0
        self.r = r
        self.T = T
        self.n = n
        self.dt = T / n
        self.model = model
        self.sigma = sigma  # For GBM, sigma is constant; for Heston, sigma is the initial volatility.

    def vol(self, sigma):
        if self.model == 'gbm':
            return np.array([sigma] * self.n)
        elif self.model == 'heston':
            # Heston model parameters
            kappa = 2.0         # speed of mean reversion
            theta = sigma**2    # long-run variance (theta)
            xi = 0.1            # volatility of volatility
            v = np.zeros(self.n)
            v[0] = sigma**2     # initial variance
            for i in range(1, self.n):
                # Euler-Maruyama update for variance
                dv = kappa * (theta - v[i-1]) * self.dt + xi * np.sqrt(max(v[i-1], 0)) * np.sqrt(self.dt) * np.random.normal()
                v[i] = v[i-1] + dv
                # Ensure non-negativity (using full truncation)
                v[i] = max(v[i], 0)
            # Return the volatility (sqrt of variance)
            return np.sqrt(v)

    def simulate(self):
        if self.model == 'gbm':
            innovations = np.random.normal(0, 1, self.n)
            stock_prices = np.zeros(self.n)
            stock_prices[0] = self.s0

            for i in range(1, self.n):
                stock_prices[i] = stock_prices[i-1] * np.exp(
                    (self.r - 0.5 * self.sigma**2) * self.dt + self.sigma * np.sqrt(self.dt) * innovations[i]
                )
            return stock_prices
        elif self.model == 'heston':
            # Use the Heston volatility path
            vol_path = self.vol(self.sigma)
            innovations = np.random.normal(0, 1, self.n)
            stock_prices = np.zeros(self.n)
            stock_prices[0] = self.s0

            for i in range(1, self.n):
                stock_prices[i] = stock_prices[i-1] * np.exp(
                    (self.r - 0.5 * vol_path[i]**2) * self.dt + vol_path[i] * np.sqrt(self.dt) * innovations[i]
                )
            return stock_prices

    def option_price(self, K):
        stock_prices = self.simulate()
        # European call option payoff at maturity
        payoff = np.maximum(stock_prices[-1] - K, 0)
        return np.exp(-self.r * self.T) * np.mean(payoff)


# Example usage:
if __name__ == "__main__":
    # Using the Heston model
    heston_stock = stock(s0=100, r=0.05, sigma=0.2, T=1.0, n=250, model='heston')
    simulated_prices = heston_stock.simulate()
    price = heston_stock.option_price(K=100)

    print("Simulated Stock Prices (Heston):")
    print(simulated_prices)
    print("\nOption Price (Heston):", price)


Simulated Stock Prices (Heston):
[100.          98.91305202  99.43182792  97.53779972  98.8724
  99.19656941 100.44336647  98.9270396   99.84395398  99.15319125
 100.8074453  102.1190933  102.11042751 105.22724871 104.58084624
 104.61359933 107.26965634 108.05314354 107.45533476 107.2164884
 107.62231566 107.47808574 105.11108455 105.79578246 103.20656039
 104.8245379  104.60738242 105.43321864 108.32742552 108.91486128
 110.2893015  108.09804227 107.31331089 107.30651256 101.71671981
 102.25448122 101.88618472 101.15174631 101.09559578 103.94466639
 102.04255386 102.11887704 102.01115269 103.09921013 102.14502885
 100.64161319 101.59005442  99.3393346  100.95439171 104.68429045
 103.99471146 103.58428115 102.77856025 100.57933053  98.96480844
  98.45224563 100.6457208  103.18071316 103.36527087 102.59838816
 101.35869774 100.80103924 101.88398991 102.85482407 102.20052193
 102.32701967  98.38308865  96.76184744  97.4527611   98.41376635
  98.20255286  98.54713218  95.32112937  96.1365